In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [3]:
credit=pd.read_csv('C:/Users/SHIVANSHU SRIVASTAVA/OneDrive/Desktop/movie recomendation system/Data_set/tmdb_5000_credits.csv')
movies=pd.read_csv('C:/Users/SHIVANSHU SRIVASTAVA/OneDrive/Desktop/movie recomendation system/Data_set/tmdb_5000_movies.csv')
movies.head(2)

In [4]:
credit.head(2)

In [5]:
movies.shape
#credit.shape

In [6]:
movies=movies.merge(credit,on='title')
movies.head(2)

In [7]:
movies.shape

In [8]:
movies['original_language'].value_counts()

In [9]:
movies.columns

In [10]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [11]:
movies.head(2)

In [12]:
movies.shape

In [13]:
movies.isnull().sum()

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.shape

In [16]:
movies.duplicated().sum()

In [17]:
movies.head(2)

In [18]:
movies.iloc[0].genres

In [19]:
import ast # convert strng to list

def convert(text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])

    return l


In [20]:
movies['genres']=movies['genres'].apply(convert)

In [21]:
movies.head(2)

In [22]:
movies.iloc[0].keywords

In [23]:
movies['keywords']=movies['keywords'].apply(convert)

In [24]:
movies.head(2)

In [25]:
movies.iloc[0].cast

In [26]:
#import ast # convert strng to list

def convert_cast(text):
    l=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter<3:
            l.append(i['name'])
        counter+=1    


    return l


In [27]:
movies['cast'] = movies['cast'].apply(convert_cast)
movies.head(2)

In [28]:
movies.iloc[0].crew

In [29]:
def fetch_director(text):
    l=[]
    for i in ast.literal_eval(text):
        if i['job'] =='Director':
            l.append(i['name'])

    return l

In [30]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head(2)

In [31]:
movies.iloc[0].overview

In [32]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()

In [33]:
movies.iloc[0].overview

In [34]:
#sam worthington ->samworthington

def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(" ", ""))
    return l    

In [35]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [36]:
movies.head()

In [37]:
movies['tags'] =movies['overview'] +movies['genres'] + movies['keywords'] + movies['cast']+ movies['crew']

In [38]:
movies.head()

In [39]:
movies.iloc[0].tags

In [40]:
new_df = movies[['movie_id' ,'title','tags']]
new_df.head()

In [41]:
new_df['tags']=new_df['tags'].apply(lambda x:' '.join(x))
new_df.head()

In [42]:
new_df.iloc[0]['tags']

In [43]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())
new_df.head()

In [44]:
new_df.iloc[0]['tags']

In [45]:
new_df.shape

In [46]:
import nltk
from nltk.stem import PorterStemmer

In [47]:
ps =PorterStemmer()

In [48]:
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))

    return " ".join(l)    

In [49]:
new_df['tags']=new_df['tags'].apply(stems)

In [50]:
new_df.iloc[0]['tags']

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [52]:
vector =cv.fit_transform(new_df['tags']).toarray()


In [53]:
vector

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
similarity = cosine_similarity(vector)

In [56]:
similarity

In [57]:
new_df[new_df['title'] =='Spider-Man'].index[0]

In [58]:
def recommend(movie):
    if movie not in new_df['title'].values:
        print(f"Movie '{movie}' not found in the dataset.")
        return
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [59]:
recommend('Spider-Man')


In [60]:
recommend('The Dark Knight Rises')

In [61]:
recommend('spectre')

In [62]:
import pickle

pickle.dump(new_df,open('artificats/movie_list.pkl','wb'))
pickle.dump(similarity,open('artificats/similarity.pkl','wb'))